# Orbit determination example
This notebook does the following:
* Download an orbit first guess from SpaceTrack
* Download laser ranging data
* Feed the data to Orekit
* Estimate the orbit
* Propagate and compare the orbit to the first guess

First, some parameters need to be defined for the orbit determination:
* Satellite ID in NORAD and COSPAR format
* Spacecraft mass: important for the drag term
* OD date: date at which the orbit will be estimated. 
* Data collection duration: for example, if equals 2 days, the laser data from the 2 days before the OD date will be used to estimate the orbit. This value is an important trade-off for the quality of the orbit determination:
    * The longer the duration, the more ranging data is available, which can increase the quality of the estimation
    * The longer the duration, the longer the orbit must be propagated, and the higher the covariance because of the orbit perturbations such as the gravity field, drag, Sun, Moon, etc.

In [1]:
# Parameters
noradId = 42829
cosparId = '1704205' 
mass = 20.0 # kg

from datetime import datetime
odDate = datetime(2018, 7, 14) # Beginning of the orbit determination
collectionDuration = 2.0 # days
from datetime import timedelta
startCollectionDate = odDate + timedelta(days=-collectionDuration)

The following sets up accounts for SpaceTrack (for orbit data) and the EDC API (for laser ranging data).
* A SpaceTrack account is required, it can be created for free at: https://www.space-track.org/auth/createAccount
* An EDC account is required, it can be created for free at: https://edc.dgfi.tum.de/en/register/

In [2]:
# Space-Track
identity_st = 'clement@jonglez.space' # Email address for Space-Track
import getpass
print('Enter SpaceTrack password for account {}'.format(identity_st))
password_st = getpass.getpass()
import spacetrack.operators as op
from spacetrack import SpaceTrackClient
st = SpaceTrackClient(identity=identity_st, password=password_st)

# EDC API
username_edc = 'jonglez' # Username for EDC API
print('Enter EDC API password for account {}'.format(username_edc))
password_edc = getpass.getpass() # You will get prompted for your password
url = 'https://edc.dgfi.tum.de/api/v1/'

Enter SpaceTrack password for account clement@jonglez.space
········
Enter EDC API password for account jonglez
········


The orbit determination needs a first guess. For this, we use Two-Line Elements. Retrieving the latest TLE prior to the beginning of the orbit determination. It is important to have a "fresh" TLE, because the newer the TLE, the better the orbit estimation.

In [3]:
rawTle = st.tle(norad_cat_id=noradId, epoch='<{}'.format(odDate), orderby='epoch desc', limit=1, format='tle')
tleLine1 = rawTle.split('\n')[0]
tleLine2 = rawTle.split('\n')[1]
print(tleLine1)
print(tleLine2)

1 42829U 17042E   18194.87572169 +.00000240 +00000-0 +29796-4 0  9990
2 42829 097.5943 096.2213 0015726 108.7371 251.5556 14.90835887054309


Looking for laser ranging data prior to the OD date.

The API only allows to look for data using the date formats 2018-07-1%, 2018-07-14% or 2018-07-14 0% for example. As a consequence, the search must be split into several days. The results are then sorted, and the observations which are outside of the date range are deleted.

In [4]:
import requests
import json
search_args = {}
search_args['username'] = username_edc
search_args['password'] = password_edc
search_args['action'] = 'data-query'
search_args['data_type'] = 'NPT' # Normal pointing data
search_args['satellite'] = cosparId

observationList = []

for i in range(int(collectionDuration) + 1): # Making a request for each day
    endDate = startCollectionDate + timedelta(days=i)
    search_args['end_data_date'] = '{:%Y-%m-%d}%'.format(endDate)
    
    search_response = requests.post(url, data=search_args)

    if search_response.status_code == 200:
        search_data = json.loads(search_response.text)
        
        for observation in search_data:
            startDataDate = datetime.strptime(observation['start_data_date'], '%Y-%m-%d %H:%M:%S')
            endDataDate = datetime.strptime(observation['end_data_date'], '%Y-%m-%d %H:%M:%S')
            
            if (startDataDate > startCollectionDate) and (endDataDate < odDate): # Only taking the values within the date range
                observationList.append(observation['id'])
            
    else:
        print(search_response.status_code)
        print(search_response.text)

Downloading the list of observations.

In [5]:
dl_args = {}
dl_args['username'] = username_edc
dl_args['password'] = password_edc
dl_args['action'] = 'data-download'
dl_args['data_type'] = 'NPT'

for obsId in observationList:
    dl_args['id'] = obsId
    dl_response = requests.post(url, data=dl_args)

    if dl_response.status_code == 200:
        """ convert json string in python list """
        dl_data = json.loads(dl_response.text)
        print(json.dumps(dl_data, indent=4, sort_keys=True))
    else:
        print(dl_response.status_code)
        print(dl_response.text)

[
    "H1 CRD 01 2018  7 12  6",
    "H2 HERL       7840 35 01 04",
    "H3 technosat   1704205 6203    42829 0 1",
    "H4  1 2018  7 12  0 27 56 2018  7 12  0 29 32  0 0 0 0 1 0 2 0",
    "C0 0   532.080   KS  khz SPD5 HMas T2L2",
    "C1 0  khz    Nd-Yag   1064.16   1000.00      1.10  10.0 20.00   1",
    "C2 0 SPD5 SPAD5        532.080 20.00   0.0   0.0      +0.7v   0.0  0.15  20.0   0.0 Single_fot",
    "C3 0 HMas iMaser_3000  iMaser_3000  HxET_=_3x_dassault   55    -0.498",
    "20  1442.880          1017.57 285.80  96. 0",
    "20  1676.160          1017.57 285.80  97. 0",
    "20  2160.000          1017.56 285.70  97. 0",
    "40 82840.320         0   KS    3058    3058  122.977   98918.4     0.0  23.3   0.1  -0.4  -1.5 2 0 0",
    "40  2229.120         0   KS    3019    3019  122.977   98917.8     0.0  22.0   0.1  -0.4  -1.5 2 0 0",
    "50   KS  38.0  0.166 -0.680   -1.0 0",
    "11  1689.9170218     0.007525834575   KS 2  15.0   123     124.0 -1.000 -1.000     -1.0  0.8 0",


[
    "h1 CRD  1 2018  7 12 17",
    "h2 YARL       7090  5 13 3 ",
    "h3 technosat   1704205 6203    42829 0 1",
    "h4  1 2018  7 12 15 20  5 2018  7 12 15 23 46  0 0 0 0 1 0 2 0",
    "c0 0  532.000 std la1 mcp ti1",
    "c1 0 la1 Nd:Yag     532.00      10.00     100.00  150.0 15.00    1              ",
    "c2 0 mcp MCP-PMT    532.000  15.5 3000.0  31.0 analog 400.0  1.00  80.0 30.00 none                  ",
    "c3 0 ti1 Truetime_XLDC Truetime_XLDC Cybi_ETM na   -1.0                                              ",
    "60  std 6 1                                                                     ",
    "40 55205.700606699997 0 std       -1       -1  -1.000    87238.0     -6.0   13.0  -1.000  -1.000   -1.0 2 2 0           ",
    "20 55209.301  985.10 284.90  85. 0                                              ",
    "11 55209.300606199999     0.005397790716 std 2   15.0     13   44.0   0.233  -0.763      -1.0   8.67 0                  ",
    "20 55222.501  985.10 284.90  85. 0 

[
    "h1 CRD  1 2018  7 13  5",
    "h2 GODL       7105  7 25 3 ",
    "h3 technosat   1704205 6203       -1 0 1",
    "h4  1 2018  7 13  3 57 21 2018  7 13  3 59  0  0 0 0 0 1 0 2 0",
    "c0 0  532.000 std la1 mcp ti1",
    "c1 0 la1 Nd:Yag     532.00      10.00     100.00  150.0 15.00    1              ",
    "c2 0 mcp MCP-PMT    532.000  12.0 2800.0  31.0 analog 400.0  1.00  80.0 30.00 none                  ",
    "c3 0 ti1 Truetime_XLDC Truetime_XLDC Cybi_ETM na   -1.0                                             ",
    "60  std 7 1                                                                     ",
    "40 14241.300531000001 0 std       -1       -1  -1.000    82511.0      0.0    9.0  -1.000  -1.000   -1.0 2 2 0           ",
    "20 14244.401 1016.40 292.40  79. 0                                              ",
    "11 14244.400532100000     0.007881081525 std 2   15.0      6   52.0  -0.371  -1.468      -1.0   4.00 0                  ",
    "20 14255.101 1016.40 292.40  79. 0  

[
    "h1 crd  1 2018  7 13 10",
    "h2       MATM 7941 77  1  4",
    "h3 technosat   1704205 6203 42829    0 1",
    "h4  1 2018  7 13  9 54  7 2018  7 13  9 54 35  0 0 0 1 1 0 2 0",
    "c0 0 532.000 std1 ml1 mcp mt1",
    "c1 0  ml1 Nd:VAN        1064.00      10.00     100.00   40.0   .00    1",
    "c2 0  mcp MCP           532.000  10.10  -1.0  -1.0 photon-dependent      -1.0   .30  50.0  -1.0       none",
    "c3 0  mt1 Symmetricom_CS4000   Meinberg_LanTime     HTSI                 na     .0",
    "11 35652.9040000072989      .0065653230944 std1 2   15.0     19      16.0    .244   -.387      -1.0  12.7 0",
    "20 35652.9040000072989  961.32 302.10  31. 0",
    "40 35652.9040000072989 0 std1 -1 -1 -1   235726.8     -1.0    6.8   -.141   -.135   -1.0 2 1 0",
    "11 35663.6040000072955      .0063594795834 std1 2   15.0     24      14.5    .820    .605      -1.0  16.0 0",
    "11 35670.4040000072894      .0062474970671 std1 2   15.0      2       2.4    .000  -2.000      -1.0   1.3

[
    "H1 CRD 01 2018 07 16 08",
    "H2 GRZL       7839 34 02  4",
    "H3 technosat   1704205 6203    42829 0 1",
    "H4  1 2018  7 13 21 32 20 2018  7 13 21 41 25  0 0 0 0 1 0 2 0",
    "C0 0 532.000 0902 2kHz C_SPAD1 GPS",
    "C1 0 2kHz Nd:Van 1064 2000 0.400 10 10 1",
    "C2 0 C_SPAD1 SPAD 532.0 20 5.0  400 +1V 10 0.3 35  300 WinClean2.2",
    "C3 0 GPS HP58503A HP58503A Graz_Dassault NoSN 0.077",
    "20 77712 961.50 293.35 68.9 1",
    "20 77907 961.50 293.32 68.9 1",
    "40 77712 0 0902 10000  7791     1.742  111902.5   0.2  14   0.009  -0.667     1 2 2 0",
    "40 77907 0 0902 10000  7876     1.742  111902.7   0.2  14   0.034  -0.662    -1 2 2 0",
    "11 77727.508563816544    0.008914071904 0902 2   15.0    122    96.9  0.053 -1.549    -90.6   0.4 0",
    "11 77737.143063683056    0.008709015166 0902 2   15.0    941    99.8  0.205 -1.673    -89.1   3.1 0",
    "11 77752.816263686864    0.008419260445 0902 2   15.0   1737    84.7  0.174 -1.475    -76.6   5.8 0",
    "11 77